In [1]:
import os
import json
import random
import pickle
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd

import paddle
from paddle.io import Dataset, DataLoader
from paddle import nn
import paddle.nn.functional as F

In [2]:
paddle.device.set_device('gpu')

Place(gpu:0)

In [3]:
def seed_paddle(seed):
    seed = int(seed)
   
    random.seed(seed) 

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed) 
    paddle.seed(seed) 

seed_paddle(seed=1024)

In [4]:
class TsDataset(Dataset):
    def __init__(self, df):

        self.seq_list1 = list(df['Wspd_seq'])
        self.seq_list2 = list(df['Patv_seq'])
        
        self.seq_list3 = list(df['Etmp_seq'])
        self.seq_list4 = list(df['Itmp_seq'])
        
        self.seq_list5 = list(df['Patv_space'])

        
        self.label_list = df.target.values

    def __getitem__(self, index):
        
        seq = np.vstack((self.seq_list1[index], self.seq_list2[index], self.seq_list3[index], self.seq_list4[index]))
        seq = np.array(seq).astype('float') 
        
        image = np.array(self.seq_list5[index]).astype('float') 
        image.resize(11, 11, 1)

        label = np.array( self.label_list[index] ).astype( 'float' )
        
        seq = paddle.to_tensor( seq )
        space_data = paddle.to_tensor(image)

        return seq, space_data, label


    def __len__(self):
        return len(self.seq_list1)

In [5]:
class GRU(nn.Layer):
    def __init__(self):
        super(GRU, self).__init__()
        
        self.gru = nn.GRU(input_size=4, hidden_size=48, num_layers=2)
        self.dropout = nn.Dropout(0.1)

        self.Linear = nn.Linear(48+2, 1, bias_attr=True)

        self.cnnLayer = nn.Sequential(
        nn.Conv2D(1, 64, kernel_size=3, stride=1, padding=1), 
        nn.BatchNorm2D(64),
        nn.GELU(),
        nn.MaxPool2D((3,3)))

        
    def forward(self, X, space_data):
        z = paddle.zeros([X.shape[0], 144, X.shape[1]], dtype="float32")
        x = paddle.concat((X.transpose([0,2,1]),z), axis=1)
        out1, _ = self.gru(x)
        ou1 = self.dropout(out1)

        cnn_out = self.cnnLayer(space_data)
        cnn_out = paddle.reshape(cnn_out, (cnn_out.shape[0], 288, -1))

        out2 = self.Linear(paddle.concat((out1, cnn_out), 2))

       
        return out2

In [6]:
model = GRU()
optimizer = paddle.optimizer.Adam(learning_rate=0.0005, parameters=model.parameters())
criterion = nn.MSELoss()

W0722 12:23:31.763727 28544 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0722 12:23:31.768137 28544 gpu_resources.cc:91] device: 0, cuDNN Version: 7.6.


In [8]:
df_train = pd.read_csv('../train_space_normal.csv')

In [9]:
cols = [x for x in df_train.columns if 'seq' in x or x=='target' or x=='Patv_space']
for col in cols:
    df_train[col] = df_train[col].apply(lambda x: json.loads(x))

In [10]:
train_dataset = TsDataset(df_train)

train_loader = DataLoader(train_dataset,
                      batch_size=128,
                      shuffle=True,
                      num_workers=4)

In [ ]:
ls = []
for epoch in range(50):
    model.train()
    pred_list = []
    label_list = []
    
    loss_list = []
    for seq, space_data, label in tqdm(train_loader):
        seq = paddle.cast(seq, dtype='float32')
        space_data = paddle.reshape(space_data, (-1,1,11,11 ))
        space_data = paddle.cast(space_data, dtype='float32')
        label = paddle.cast(label, dtype='float32')
        
        pred = model(seq, space_data)
        
        loss = criterion(pred.squeeze()/1000, label/1000)
        loss_list.append(loss)
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        
        pred_list.extend(pred.squeeze().cpu().detach().numpy())
        label_list.extend(label.squeeze().cpu().detach().numpy())
        del pred 
        del seq
        del space_data
        del label
    
    total_loss = paddle.mean(paddle.to_tensor(loss_list))
    model.eval()

    print(
        f'Epoch: {epoch} Loss: {total_loss}'
    )
    

  0%|          | 0/340 [00:00<?, ?it/s]/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:654: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
  0%|          | 0/340 [00:00<?, ?it/s]

Epoch: 0 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.30510363])


  0%|          | 0/340 [00:00<?, ?it/s]

Epoch: 1 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.29717985])


  0%|          | 0/340 [00:00<?, ?it/s]

...

  0%|          | 0/340 [00:00<?, ?it/s]

Epoch: 7 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.25604886])


  0%|          | 0/340 [00:00<?, ?it/s]

Epoch: 8 Loss: Tensor(shape=[1], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [0.24948820])


 81%|████████  | 274/340 [00:05<00:01, 52.35it/s]

In [19]:
#paddle.save(model.state_dict(), "paddle_gru.pdparams")